#  The LightCurve class

##  Background
#### What kind of physical data are we representing and what do these quantities mean?

Astrophysical variable sources including transients like supernovae are sources that are (roughly) fixed in position but change appreciably at human time scales, such that their measurements at different points of time in a survey are expected to be different. The key science of such sources has to do with interpreting such data.

In a photometric or imaging survey, the raw data associated with such measurements is in the form of a time series of images, along with metadata. This is often extracted into a catalog summarizing the inferred brightness of the source in the filter band and the uncertainties of the inferred. The uncertainties in this process are dominated by the sky noise which is Poisson distributed to a good approximation. Hence, the format of this recorded data is usually of the form

`timestamp`, `flux`, `flux uncertainty`, `zero point`, `magnitude system`

Note that the `flux` is really the band flux obtained in a filter band

The `zero point`, and `magnitude system` are methods of specifying the units of the reported `flux`, and `flux uncertainty`. `flux` may be reported so that the following equation with `bandflux` and `std_bandflux` in the same physical units holds. The `std_bandflux` depends on the transmissions and the magnitude system used.

bandmag := -2.5 log10(bandflux / std_bandflux) = -2.5 log10(flux) + zero point

so that a zero point of zero, implies that `flux` is `bandflux` in units of `std_bandflux`

## Desired Functionality

the `LightCurve` class is meant to be a standard internal representation of this kind of catalog data in our software. This means that 
- given a different format/model of data, we will first convert it into an instance of this class.
- All our calculations will be defined on instances of this class
- We should have write methods to convert the instances of this class to useful formats for serialization.

## Structure

In [1]:
import sncosmo
#import analyzeSN as ans
import sndata as snd
import numpy as np
import sndata
print(sndata.__file__)

This requires a THROUGHPUTS directory to be setup as an env var

This may be achieved, for example, by setting up the LSST stack

or by cloning the throughputs directory from git-lfs, 

and setting up env vars. The function is to provide the LSST bandpass

files

/Users/rbiswas/doc/projects/DESC/SNData/sndata/__init__.pyc


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

#### Instantiation

The class is instantiated using a constructor that has a mandatory input of a dataframe. and optional inputs. The dataframe must contain the light curve information, and the minimum set of columns required are:
`mjd`, `band`, `flux`, `flux_err`, `zp`, `zpsys` or recognized aliases. Here, we load some data and demonstrate this

In [3]:
ex_data = sncosmo.load_example_data().to_pandas()
ex_data.head()

,time,band,flux,fluxerr,zp,zpsys
0,55070.000000,sdssg,0.363512,0.672844,25.0,ab
1,55072.051282,sdssr,-0.200801,0.672844,25.0,ab
2,55074.102564,sdssi,0.307494,0.672844,25.0,ab
3,55076.153846,sdssz,1.087761,0.672844,25.0,ab
4,55078.205128,sdssg,-0.436679,0.672844,25.0,ab


This was a SNCosmo example dataset, loaded into `pandas.DataFrame`. Note that the column representing temporal information is `time`. Now, we will use this to instantiate our class.

In [4]:
lc = snd.LightCurve(ex_data)

We get the representation in our convention by accessing the attribute `lightCurve`, which is a dataFrame with columns of the standard name. It could have more columns, but the minimal columns have to be there. If we supplied a dataFrame without having the minimal number of columns, it will raise a `valueError`, as described later. From the `lightCurve` attribute, we note that `time` has changed to our standard name `mjd`

In [5]:
lc.lightCurve.head()

,mjd,band,flux,fluxerr,zp,zpsys
0,55070.000000,sdssg,0.363512,0.672844,25.0,ab
1,55072.051282,sdssr,-0.200801,0.672844,25.0,ab
2,55074.102564,sdssi,0.307494,0.672844,25.0,ab
3,55076.153846,sdssz,1.087761,0.672844,25.0,ab
4,55078.205128,sdssg,-0.436679,0.672844,25.0,ab


What would have happened if an essential column was missing? to check let us do the following:
   - We create a dataframe like the above, but remove the essential column band
   - We attempt to create an instance of the LightCurve class

In [14]:
tmp_data = ex_data.copy()
del tmp_data['band']
tmp_data.head()

,mjd,flux,fluxerr,zp,zpsys
0,55070.000000,0.363512,0.672844,25.0,ab
1,55072.051282,-0.200801,0.672844,25.0,ab
2,55074.102564,0.307494,0.672844,25.0,ab
3,55076.153846,1.087761,0.672844,25.0,ab
4,55078.205128,-0.436679,0.672844,25.0,ab


In [15]:
lc_bad = snd.LightCurve(tmp_data)

ValueError: ('light curve data has missing columns', set(['band']))

This raises a ValueError pointing out that the column `band` was missing. However, it correctly recognizes that while
`mjd` was missing, it had the aliased column `time` instead and did not report it missing.

## Coadded light Curves

In [16]:
lc.lightCurve.head()

,mjd,band,flux,fluxerr,zp,zpsys
0,55070.000000,sdssg,0.363512,0.672844,25.0,ab
1,55072.051282,sdssr,-0.200801,0.672844,25.0,ab
2,55074.102564,sdssi,0.307494,0.672844,25.0,ab
3,55076.153846,sdssz,1.087761,0.672844,25.0,ab
4,55078.205128,sdssg,-0.436679,0.672844,25.0,ab


Let us check the cadence on the single band light curve. These have a cadence of about ~ 8 days

In [17]:
lc.lightCurve.groupby('band').get_group('sdssg')

,mjd,band,flux,fluxerr,zp,zpsys
0,55070.000000,sdssg,0.363512,0.672844,25.0,ab
4,55078.205128,sdssg,-0.436679,0.672844,25.0,ab
8,55086.410256,sdssg,2.826142,0.672844,25.0,ab
12,55094.615385,sdssg,1.578237,0.672844,25.0,ab
16,55102.820513,sdssg,2.208754,0.672844,25.0,ab
20,55111.025641,sdssg,0.608391,0.672844,25.0,ab
24,55119.230769,sdssg,0.679196,0.672844,25.0,ab
28,55127.435897,sdssg,0.325443,0.672844,25.0,ab
32,55135.641026,sdssg,0.330702,0.672844,25.0,ab
36,55143.846154,sdssg,-0.722655,0.672844,25.0,ab


In [18]:
lc.coaddedLC(coaddTimes=10.0).numExpinCoadd.describe()

count    33.000000
mean      1.212121
std       0.415149
min       1.000000
25%       1.000000
50%       1.000000
75%       1.000000
max       2.000000
Name: numExpinCoadd, dtype: float64

In [19]:
lc.coaddedLC(coaddTimes=10.0).head()

,band,night,zpsys,numExpinCoadd,flux,mjd,zp,fluxerr
0,sdssg,5507,ab,2,-0.036584,55074.102564,25.0,0.475772
1,sdssg,5508,ab,1,2.826142,55086.410256,25.0,0.672844
2,sdssg,5509,ab,1,1.578237,55094.615385,25.0,0.672844
3,sdssg,5510,ab,1,2.208754,55102.820513,25.0,0.672844
4,sdssg,5511,ab,2,0.643794,55115.128205,25.0,0.475772


In [20]:
exlc = lc.lightCurve.query('band == "sdssg"').iloc[:2]

With the few observations that are being coadded, we can check the results of a invariance variance weighting by hand

In [21]:
w = 1.0 / exlc.fluxerr**2

In [22]:
# For Flux values
np.average(exlc.flux.values, weights=w)

-0.036583710239999997

In [23]:
lc.coaddedLC(coaddTimes=10.0).flux.iloc[0]

-0.036583710239999997

In [24]:
np.average(exlc.mjd.values, weights=w)

55074.102564100001

In [25]:
np.average(exlc.zp.values, weights=w)

25.0

In [26]:
np.sqrt(1.0 / w.sum())

0.47577244727588863

In [27]:
lc._coaddedLC(coaddTimes=10.)

,band,mjd,fluxerr,zpsys,flux,numCoadded,zp,CoaddedSNR
discreteTime,,,,,,,,
0,sdssg,55074.102564,0.475772,ab,-0.036584,2,25.0,-0.076893
0,sdssi,55074.102564,0.672844,ab,0.307494,1,25.0,0.457007
0,sdssr,55072.051282,0.672844,ab,-0.200801,1,25.0,-0.298437
0,sdssz,55076.153846,0.672844,ab,1.087761,1,25.0,1.616662
1,sdssg,55086.410256,0.672844,ab,2.826142,1,25.0,4.200294
1,sdssi,55082.307692,0.672844,ab,3.756269,1,25.0,5.582675
1,sdssr,55084.358974,0.475772,ab,4.331640,2,25.0,9.104437
1,sdssz,55084.358974,0.672844,ab,5.348589,1,25.0,7.949228
2,sdssg,55094.615385,0.672844,ab,1.578237,1,25.0,2.345621


## Summarizing Light Curve information

In [6]:
df = lc.lightCurve
df['snid'] = 0

In [7]:
import pandas as pd

In [8]:
paramsdf = pd.DataFrame(dict(snid=(0,), t0=(55140,)))

In [9]:
x = lc.summarize(df, grouping=('snid', 'band'), paramsdf=paramsdf)

In [10]:
x

,snid,t0,mjd_max_sdssg,mjd_max_sdssi,mjd_max_sdssr,mjd_max_sdssz,mjd_min_sdssg,mjd_min_sdssi,mjd_min_sdssr,mjd_min_sdssz,NOBS_sdssg,NOBS_sdssi,NOBS_sdssr,NOBS_sdssz,SNR_max_sdssg,SNR_max_sdssi,SNR_max_sdssr,SNR_max_sdssz
0,0,55140,55143.846154,55147.948718,55145.897436,55150.0,55070.0,55074.102564,55072.051282,55076.153846,10,10,10,10,4.200294,18.000641,14.318954,20.278901


In [33]:
x = lc.summarize(df, grouping=('snid',), paramsdf=paramsdf)

In [34]:
x

,snid,t0,mjd_max,mjd_min,NOBS,SNR_max
0,0,55140,55150.0,55070.0,40,20.278901


### Light Curves in SNCosmo Format

SNCosmo has a data format using `astropy.table.Table`. Since, we use SNCosmo to plot, or fit light curves, it is essential that we can regenerate the SNCosmo format data easily. We do this by the following method

In [ ]:
SNCosmoLC = lc.snCosmoLC()

In [ ]:
SNCosmoLC

We can now plot the data using the usual SNCosmo plotting method, showing the data in g, r, and z bands.

In [ ]:
sncosmo.plot_lc(SNCosmoLC,color='k');

## SNANA simulations

In [ ]:
from sndata import SNANASims

In [ ]:
from astropy.table import Table

In [ ]:
megacamBandNames = 'ugriz'
#megacamRegisteredNames = tuple('megacam' + band for band in megacamBandNames)
megacamRegisteredNames = tuple('megacam' + band for band in megacamBandNames)

In [ ]:
snana_eg = SNANASims.fromSNANAfileroot(snanafileroot='snana_fits',
                                       location='../sndata/example_data/',
                                       coerce_inds2int=False,
                                       SNANABandNames=megacamBandNames,
                                       registeredBandNames=megacamRegisteredNames)

In [ ]:
Table.read(snana_eg.photFile)[45:50]

Let us look at what the array in the fits file looks like: 
- It has fields that are not of the standard name (FLUXCAL)
- It has things like "XXX" in he fields

In [ ]:
snana_eg.bandNames

In [ ]:
snana_eg.newbandNames

In [ ]:
lcInstance = snana_eg.get_SNANA_photometry(snid='03D1aw')

In [ ]:
lcInstance.lightCurve[['mjd', 'band', 'flux', 'fluxerr', 'zp', 'zpsys']]

In [ ]:
lcInstance.bandNameDict

In [ ]:
snana_eg.headData['REDSHIFT_FINAL']

In [ ]:
snanalc = snd.LightCurve(lcInstance.lightCurve[['mjd', 'band', 'flux', 'fluxerr', 'zp', 'zpsys']])
SNCosmo_snanalc = snanalc.snCosmoLC()

In [ ]:
SNCosmo_snanalc

In [ ]:
sncosmo.plot_lc(SNCosmo_snanalc);#, color='k');

## Fitting

import sncosmo

To infer the light curve model parameters, we need a model. We will use the SALT model from SNCosmo

In [ ]:
dust = sncosmo.CCM89Dust()
print(dust)

In [ ]:
model = sncosmo.Model(source='salt2', effects=[dust,dust],effect_names=['host', 'mw'], effect_frames=['rest', 'obs'])
#print(model)
#model.param_names
#model.parameters

We will not try to infer the redshift first

In [ ]:
model.set(z=0.5817)

We have two main methods of trying to infer the model parameters: the first one is a maximum likelihood estimate and second one is a Markov Chain Monte Carlo

In [ ]:
resml,fitmodml = sncosmo.fit_lc(lcInstance.snCosmoLC(), model, vparam_names=['t0', 'x0', 'x1', 'c'], modelcov=False)
resmcmc,fitmodmcmc = sncosmo.mcmc_lc(lcInstance.snCosmoLC(), model, vparam_names=['t0', 'x0', 'x1', 'c'], modelcov=False)

Get results from fits using sncosmo

In [ ]:
print(resmcmc)
print(fitmodml)
print(resml.parameters)

Plotting LC for maximum likelihood model

In [ ]:
sncosmo.plot_lc(lcInstance.snCosmoLC(), model=fitmodml, errors=resml.errors);

Plotting LC for MCMC

In [ ]:
sncosmo.plot_lc(lcInstance.snCosmoLC(), model=fitmodmcmc, errors=resmcmc.errors);

In [ ]:
fig, ax  = plt.subplots(1, 4,figsize=(18, 8))
for i in range(0,4):
    ax[i].hist(resmcmc.samples[:,i],bins=15, normed=1);
    ax[i].set_xlabel(resmcmc.vparam_names[i])
#resmcmc.samples()['t0'].hist(bins=10)#, lw=2., histtype='step', normed=1, ax=ax[2])
#resmcmc.samples()['x0'].hist(bins=10, lw=2., histtype='step', normed=1, ax=ax[3])
#resmcmc.samples()['x1'].hist(bins=np.arange(-4., 4., 0.2), lw=2., histtype='step', normed=1, ax=ax[1])
#resmcmc.samples()['c'].hist(bins=np.arange(-0.6, 0.6, 0.03), lw=2., histtype='step', normed=1, ax=ax[0])

Plotting the LC of the previous example

In [ ]:
resfit_old,fitmod_old = sncosmo.fit_lc(lc.snCosmoLC(), model, vparam_names=['z','t0', 'x0', 'x1', 'c'],bounds={'z':(0.0, 0.7)}, modelcov=False)
sncosmo.plot_lc(lc.snCosmoLC(), model=fitmod_old, errors=resfit_old.errors);

In [ ]:
resmcmc_old,fitmodmcmc_old = sncosmo.mcmc_lc(lc.snCosmoLC(), model, vparam_names=['z','t0', 'x0', 'x1', 'c'],bounds={'z':(0.0, 0.7)}, modelcov=False)
sncosmo.plot_lc(lc.snCosmoLC(), model=fitmodmcmc_old, errors=resmcmc_old.errors);